In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from tqdm import tqdm_notebook, tqdm
from tqdm._tqdm_notebook import tqdm_notebook
import time
import warnings
warnings.filterwarnings(action='ignore')
from collections import OrderedDict, defaultdict
import subprocess
import json
import csv
import gc
from itertools import chain, islice
import itertools
import os


from typing import Dict #dict형식 
from collections import Counter #동일한값의 자료가 몇개인지 파악하는데 사용

#import missingno as msno
#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

pd.options.display.max_columns = 900

# Data load

In [ ]:
# #CSV
# path = './001.data/'
# train_df = pd.read_csv(path + 'train.csv')
# test_df = pd.read_csv(path + 'test.csv')
# train_labels_df = pd.read_csv(path +'train_labels.csv')
# sample_submission = pd.read_csv(path +'sample_submission.csv')
# train_df.shape, test_df.shape,train_labels_df.shape

In [ ]:
path = '/kaggle/input/data-science-bowl-2019/'
train_df = pd.read_csv(path+'train.csv')
test_df = pd.read_csv(path+'test.csv')
train_labels_df = pd.read_csv(path+'train_labels.csv')
sample_submission = pd.read_csv(path+'sample_submission.csv')

# user func

In [ ]:
def info_df(data):
    '''
    data의 type, null_count, null_rate를 알려주는 함수 
    추가로 unique한 데이터의 수를 알려줌 (속도가 느리므로 필요시 주석처리 필요)
    '''
    info_df = pd.DataFrame({"type":data.dtypes,
                            'null_count':data.isnull().sum(),
                           'null_rate':data.isnull().sum()/data.isnull().count() * 100})
    
#     cols = data.columns.values
    
#     uni_count =[]
#     for col in cols:
#         uni_count.append(len(data[col].unique()))
#     info_df['uni_count'] = uni_count
    
    return info_df

In [ ]:
#feature importance
def model_fi(model,origin_df):
    '''
    모델의 feature_importance를 확인하여, origin_df의 컬럼명에 따른 DF생성
    importance의 비율 및 누적합 또한 산출   
    '''    
    fi_df = pd.DataFrame({'importance': model.feature_importances_},index = origin_df.columns)
    fi_df = fi_df.sort_values(by ='importance', ascending=False)
    fi_df['importance'] = round(fi_df['importance'],4)
    fi_df['rate'] = (fi_df.importance/fi_df.importance.sum()*100)
    fi_df['rate_cumsum'] = fi_df.rate.cumsum()
    
    return fi_df

### cut_train 재이해
* 기존코드 짧게 가능한듯 
* 마지막 Assessment 이후처리가 필요한가? 어차피 for문 돌릴거라면 굳이? 할필요없다 (왜냐면 오래걸리니까)

#유니크한값만
uni_inst = train_labels_df.installation_id.unique()
#들어오는거 안들어오는거
inlabel = train_df[train_df.installation_id.isin(list(uni_inst))]
inlabel = train_df[~train_df.installation_id.isin(list(uni_inst))]

#들어오지않는것중 승리코드가 있는 경우는 없다
outlabel[(outlabel.type == 'Assessment')&(outlabel.event_code == 4110)].shape, outlabel[(outlabel.type == 'Assessment')&(outlabel.event_code == 4100)].shape

In [ ]:
# data_mid = pd.DataFrame()
# for ins_id, user_sample in tqdm_notebook(train_s.iloc[:57].groupby('installation_id', sort = False)):
#     as_indexs = user_sample.loc[(user_sample.type.isin(['Assessment']))&
#                                 (user_sample.accuracy_group.notnull())].index.values
#     last_index = as_indexs[-1]
# #     last_index = user_sample.index.max()
#     sess_df = pd.DataFrame()
#     #user_sample.drop(columns='accuracy_group', axis=1, inplace=True)
#     for as_index in as_indexs:
#         #iloc를 쓰면 as_index에 +1 
#         #loc를 쓰면 as_index그대로 사용
#         session = user_sample.iloc[n:as_index+1,:]
#         session_df = session.groupby('installation_id').sum().reset_index(drop=True)
#         sess_df = pd.concat([sess_df, session_df])
#     data_mid = pd.concat([data_mid, sess_df])
#     n = last_index+1
# data_mid = data_mid.reset_index(drop=True)
# data_fin = pd.concat([data_fin, data_mid], axis=1)

### 아이디어

# day gap 
first_day = sameple_df['timestamp'].iloc[0]

#game_session단위에서 
last_day = session['timestamp'].iloc[-1]
day_gap = (last_day-first_day).days

### 튜닝중

In [ ]:
#함수빌려오기 
def encode_title(train, test, train_labels):
    
    #train in train_labels
    uni_inst = train_labels.installation_id.unique()
    train = train[train.installation_id.isin(list(uni_inst))]
    
    # title_event_code 
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    
    # title, event_code, event_id, world
    list_of_title = list(set(train['title'].unique()).union(set(test['title'].unique())))
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    list_of_worlds = list(set(train['world'].unique()))   
    
    # create a dictionary numerating the titles
    title_labels = dict(zip(list_of_title, np.arange(len(list_of_title)))) 
    labels_title = dict(zip(np.arange(len(list_of_title)), list_of_title)) 
    world_labels = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    labels_world = dict(zip(np.arange(len(list_of_worlds)),list_of_worlds))
    assess_titles = list(train_df[train_df['type'] == 'Assessment']['title'].unique())
    
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(title_labels)
    test['title'] = test['title'].map(title_labels)
    train['world'] = train['world'].map(world_labels)
    test['world'] = test['world'].map(world_labels)
    train_labels['title'] = train_labels['title'].map(title_labels)
    
    # win_code
    #'Bird Measurer (Assessment)' is win_code is 4110 
    win_code = dict(zip(title_labels.values(), (4100*np.ones(len(title_labels))).astype('int')))  
    win_code[title_labels['Bird Measurer (Assessment)']] = 4110
    
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    
    return train, test, train_labels, all_title_event_code, list_of_title, list_of_event_code, list_of_event_id, list_of_worlds, title_labels, labels_title ,world_labels, labels_world, assess_titles, win_code

In [ ]:
train_df, test_df, train_labels_df, all_title_event_code, list_of_title, list_of_event_code, list_of_event_id, list_of_worlds, title_labels, labels_title ,world_labels, labels_world, assess_titles, win_code = encode_title(train_df, test_df, train_labels_df)

In [ ]:
labels_world

### step 등등 추가하기

In [ ]:
def get_key(string, key):
    import re
    repattern = re.compile(f'(?<="{key}":)' + '[A-z0-9\"\'\[\],]+' + "(?=[\},])")

    try:
        value = repattern.search(string).group()
    except:
        value = np.NaN
    
    return value

def add_col(data):
#     #duration (확인이 어려운부분이 있어서 생략)
#     data['duration'] =data.event_data.apply(lambda x : get_key(x,"duration")).astype(float)
#     #Scrub-A-Dub 2030은 np.nan부여
#     data.loc[(data.event_code.isin([2030]))&(data.title.isin([title_labels['Scrub-A-Dub']])),'duration'] = np.nan
#     #2050 임시 저장 
#     saved = data.loc[(data.event_code.isin([2050]))&(data.title.isin([title_labels['Scrub-A-Dub']])),'duration']
#     #2030 제외 np.nan부여
#     data.loc[data.event_code != 2030,'duration'] = np.nan 
#     data.loc[saved.index.values,'duration'] = saved
#     data.duration.fillna(0, inplace=True)
    start = dt.now()
    #step
    key_list = ["level", "round", "stage_number", "round_number"]
    for key in key_list:
        data[key] = data.event_data.apply(lambda x : get_key(x,key))
        
    #step으로 통합  
    data['step'] = data['round'].values
    data.at[data['step'].isnull(),'step'] = data.loc[data['step'].isnull(),'stage_number']
    data.at[data['step'].isnull(),'step'] = data.loc[data['step'].isnull(),'round_number']
    data.at[data['level'].notnull(),'step'] = data.loc[data['level'].notnull(),'level']
    data.drop(key_list,axis=1,inplace = True)
    data.step.fillna(0, inplace=True)
    data['step'] = data['step'].astype(int)
    
    #Assessment 제외
    label_list = []
    for title in assess_titles:
        label_list.append(title_labels[title])
    data.loc[data.title.isin(label_list),'step'] = 0
    
    step_max = data.groupby('game_session')['step'].max().reset_index()
    data.drop('step',axis= 1,inplace = True)
    data = pd.merge(data,step_max)
    
    end1 = dt.now()
    print("step 소요시간",round((end1 - start).total_seconds()/60,2),"분")
        
    #miss
    data["correct"] = data.event_data.apply(lambda x : get_key(x,"correct"))
    data.loc[data.event_code != 4020,"correct"] = np.nan
    data['cor_t'] = 0
    data['cor_f'] = 0
    data.loc[data.correct.isin(["true"]), 'cor_t'] = 1
    data.loc[data.correct.isin(["false"]), 'cor_f'] = 1
    data.drop('correct',axis=1,inplace = True)
    
    end2 = dt.now()
    print("miss 소요시간",round((end2 - end1).total_seconds()/60,2),"분")
    
    return data

### step 고려사항
* 1. 한Game_Session에 max(step)이 들어가는게 맞다
* 2. 단 게임에 따라 max가 정해져 있는 경우가 있고 아닌 경우가 있다 파악 필요
* 3. 각 게임별로 step이 다른데 이걸 하나도 뭉치기도 애매하고 하자니 품이 많이 들어가는데 그정도 가치가 있는지 모르겠어서 일단 패스

In [ ]:
#일단 패스 못해먹겟다
# test_df = add_col(test_df)
# train_df = add_col(train_df)

In [ ]:
# ori_sample[ori_sample.title == 18]['step'].describe()

In [ ]:
# #title별로 묶어서 진행
# step_title = dict(zip(step_title,[0]* len(step_title)))

# # step_title[session['title']] session['step'][0]

In [ ]:
# # step이 있는 타이틀 총 12개 (assessment 제외)
# for i in a[a>0].index.values[:]:
#     print(i, activities_labels[i])

In [ ]:
# ori_sample[ori_sample.title == 18]['step'].plot.box()

In [ ]:
# train_df.head()

### 수정중

In [ ]:
title_count: Dict[str, int] = {eve: 0 for eve in labels_title.values()} 

In [ ]:
labels_title

In [ ]:
world_count : Dict[str, int]  = {eve: 0 for eve in labels_world.values()}

In [ ]:
world_count

In [ ]:
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in labels_title.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    world_count : Dict[str, int]  = {eve: 0 for eve in labels_world.values()}
        
    # last features
    sessions_count = 0
        
    #add feature
    step =[]
    cor_t =0
    cor_f = 0
    day_gap =0
    
    #first_day of installation_id
    first_day = user_sample.iloc[0,2]
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = labels_title[session_title]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        # 타입이 Assessment이면서 test_set이 True 또는 길이가 1보다 클때
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            #승리코드에 해당하는 부분 출력 (여러번일수 있으니 데이터 프레임으로)
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            #시도에 대한 결과값을 각각 보존
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            features.update(world_count.copy())
            features['installation_session_count'] = sessions_count
            
            variety_features = [('var_event_code', event_code_count),
                              ('var_event_id', event_id_count),
                               ('var_title', title_count),
                               ('var_title_event_code', title_event_code_count),
                               ('va_title_world',world_count)]
            
            for name, dict_counts in variety_features:
                arr = np.array(list(dict_counts.values()))
                features[name] = np.count_nonzero(arr)
                 
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['game_session'] = session['game_session'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
                features['duration_std'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
            
            
            #마지막시간에서 처음시간 빼고 초로 변환 후 durations에 전달
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            day_gap = (session.iloc[0, 2] - first_day).days
            features['day_gap'] = day_gap
            
            # the accurace is the all time wins divided by the all time attempts
            #누적 어큐레시 카운트가 0이상일때만 계산
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            #어큐레시 직접계산
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            #누적값생성
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            #어큐레시 그룹누적
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            #누적 엑션수 즉 누적 row수
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            # 테스트셋의 경우 모든 피쳐를 사용하고 train의 경우에는 4100,4110가 있어야함 아니면 pass
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        sessions_count += 1
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = labels_title[k]
                    if col == 'world':
                        x = labels_world[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')
        world_count = update_counters(world_count, 'world')

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [ ]:
#위에서 정의된 get_data가 get_train_and_test함수에서 사용됨
#이함수는 installation 단위의 연산이 이뤄짐
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    for ins_id, user_sample in tqdm_notebook(train.groupby('installation_id', sort = False),total = 3614):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm_notebook(test.groupby('installation_id', sort = False), total = 1000):
        test_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    categoricals = ['session_title']
    return reduce_train, reduce_test, categoricals

In [ ]:
reduce_train, reduce_test, categoricals = get_train_and_test(train_df, test_df)

In [ ]:
#제이슨오류방지
reduce_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_train.columns]
reduce_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in reduce_train.columns]

In [ ]:
reduce_train.head()

In [ ]:
def add_feature2(data):
    '''
    현재에 넣기위해 다소 수정함 
    load했다면 숫자변수 문자로 수정해야함
    
    test_df가 있어야 작동
    #1.RAR(Right_action_rate) :잘한학습과 잘못한 학습의 비율
        * event_code 4020은 잘한 학습이며, 이는 correct: True/False로 나타남
        * event_code 4035 및 4070은 학습과정에서 잘못된 곳을 누르거나 drop한 경우
        * 4020_count/ (4035,4070_count)
    # 2. acum_winning(누적_성공률)
        * 3020/3021은 실패/성공시 나오는 message (Game/Assessment에서만 발동)
        * 3021_count/(3020,3021_count)
    # 3.same_world_rate(동일 world 학습률) :last_Assessment와 동일한 월드 횟수
        * last_title에 따른 last_world 추출 : last_world
        * installation_id당 전체 플레이한 title수 : total_title
        * 각 isntallation_id당 동일 world 플레이한 비율 : same_world_rate
        * installation_id의 last_world play횟수 / total_title 
    # 4.월드_type별 달성률
    '''
    
    # 1.RAR(Right_action_rate)
    data['RAR'] = data[4020]/(data[4035] + data[4070]+data[4020])
    data.RAR.fillna(0, inplace=True)
    
    # 2. acum_winning(누적_성공률)
    data['acum_winning'] = data[3021]/(data[3020]+data[3021])
    data.acum_winning.fillna(0, inplace=True)
    
#     # 3. same_world_rate(동일 world 학습률)
    # world_title을 합쳐놓은 df
    world_title = test_ori[['title','world']].drop_duplicates()
    world_title['title'] = world_title['title'].map(title_labels)
    
    data['last_world'] = np.nan
    for title in list(set(data.session_title)):   
        assessment_world = dict(world_title.values)
        data.loc[data.session_title == title,'last_world'] = assessment_world[title]
   
    # total_title
    data['total_title'] = data['MAGMAPEAK'] +data['CRYSTALCAVES'] +data['TREETOPCITY']
    # same_world_rate 
    same_world_rate = []
    for i in range(len(data)):
        name = data['last_world'].iloc[i]
        rate = data[name].iloc[i]/data['total_title'].iloc[i]
        same_world_rate.append(rate)
    data['same_world_rate'] = same_world_rate
    
    # 4.월드_type별 달성률
    #target은 각월드별 type이 tn과 일치하는 title명을 array로 받음
    #따라서 world_type = plat한 타이틀 수 / type별 총 타이틀수 
    world_name = ['CRYSTALCAVES','MAGMAPEAK','TREETOPCITY']
    type_name = ['Activity','Clip','Game','Assessment']
    
    title_type = test_ori[['world','title','type']].drop_duplicates('title')
    #월드_type 형태로 컬럼생성
    for wn in world_name:
        for tn in type_name:
            target = title_type.loc[(title_type['world'] == wn)&(title_type['type']== tn),'title'].values
            data[wn+"_"+tn] = (data[target]>0).apply(sum,axis=1)/len(data[target].columns)
            
    data.drop(['last_world','total_title'], axis=1, inplace=True)
    
    return data

In [ ]:
test_ori = pd.read_csv(path + 'test.csv')
reduce_train = add_feature2(reduce_train)
reduce_test= add_feature2(reduce_test)
del test_ori

In [ ]:
reduce_train.shape, reduce_test.shape

### 기본모델

In [ ]:
m_y = reduce_train.accuracy_group
m_X = reduce_train.drop(["accuracy_group","game_session","installation_id"], axis = 1)
m_test_X = reduce_test.drop(["accuracy_group","game_session","installation_id"], axis = 1)

In [ ]:
import lightgbm as lgbm

In [ ]:
default_lgbmr = lgbm.LGBMRegressor()
default_lgbmr.fit(m_X, m_y)
m_pred = default_lgbmr.predict(m_X)

In [ ]:
model_fi(default_lgbmr,m_X).head(20)

In [ ]:
dist = Counter(reduce_train['accuracy_group'])
for k in dist:
    dist[k] /= len(reduce_train)
reduce_train['accuracy_group'].hist()

acum = 0
bound = {}
for i in range(3):
    acum += dist[i]
    bound[i] = np.percentile(m_pred, acum * 100)
print(bound)

def classify(x):
    if x <= bound[0]:
        return 0
    elif x <= bound[1]:
        return 1
    elif x <= bound[2]:
        return 2
    else:
        return 3
    
final_pred = np.array(list(map(classify, m_pred)))

In [ ]:
pd.Series(final_pred).value_counts()

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(final_pred,reduce_train['accuracy_group'],weights="quadratic")

### test_data(제출용)

In [ ]:
r_m_pred = default_lgbmr.predict(m_test_X)

In [ ]:
dist = Counter(reduce_train['accuracy_group'])
for k in dist:
    dist[k] /= len(reduce_train)
reduce_train['accuracy_group'].hist()

acum = 0
bound = {}
for i in range(3):
    acum += dist[i]
    bound[i] = np.percentile(r_m_pred, acum * 100)
print(bound)

def classify(x):
    if x <= bound[0]:
        return 0
    elif x <= bound[1]:
        return 1
    elif x <= bound[2]:
        return 2
    else:
        return 3
    
final_pred = np.array(list(map(classify, r_m_pred)))

In [ ]:
pd.Series(final_pred).value_counts()

In [ ]:
sample_submission['accuracy_group'] = final_pred.astype(int)
sample_submission.to_csv('submission.csv', index=False)

### Check1 마지막 assessment는 들어갔는가?
* 결론 : 들어가지 않았다. 왜냐하면 test셋의 마지막 assessment에 대한 정보가 없기 떄문  
--> 아마 이로 인해서 overfitting이 났을것으로 추정(이미 답을알고 있기 때문에)

* 추정과정 : '0006a69f'을 대상으로 처음 game_session으로 확인   
--> 처음 : 647건, 마지막 : 2568건 확인

In [ ]:
train[train.installation_id == '0006a69f']['game_session'].nunique()
tt = train[train.installation_id == '0006a69f']

tt.shape

In [ ]:
#Assessment인거만 추출
only_ass = pd.DataFrame(tt[tt.type == 'Assessment'][['game_session','title']].drop_duplicates())
only_ass

In [ ]:
last_ass_index = tt[tt.game_session == '901acc108f55a5a1'].iloc[-1:].index[0]
first_ass_index = tt[tt.game_session == '901acc108f55a5a1'].iloc[:1].index[0]
tt.loc[:first_ass_index-1].shape

In [ ]:
last_ass_index = tt[tt.game_session == 'a9ef3ecb3d1acc6a'].iloc[-1:].index[0]
first_ass_index = tt[tt.game_session == 'a9ef3ecb3d1acc6a'].iloc[:1].index[0]
tt.loc[:first_ass_index-1].shape

### 1개 installation_id에서 확인

In [ ]:
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
        
    # last features
    sessions_count = 0
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        # 타입이 Assessment이면서 test_set이 True 또는 길이가 1보다 클때
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            #승리코드에 해당하는 부분 출력 (여러번일수 있으니 데이터 프레임으로)
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            #시도에 대한 결과값을 각각 보존
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            features['installation_session_count'] = sessions_count
            
            variety_features = [('var_event_code', event_code_count),
                              ('var_event_id', event_id_count),
                               ('var_title', title_count),
                               ('var_title_event_code', title_event_code_count)]
            
            for name, dict_counts in variety_features:
                arr = np.array(list(dict_counts.values()))
                features[name] = np.count_nonzero(arr)
                 
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
                features['duration_std'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
                features['duration_std'] = np.std(durations)
            #마지막시간에서 처음시간 빼고 초로 변환 후 durations에 전달
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            #누적 어큐레시 카운트가 0이상일때만 계산
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            #어큐레시 직접계산
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            #누적값생성
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            #어큐레시 그룹누적
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            #누적 엑션수 즉 누적 row수
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            # 테스트셋의 경우 모든 피쳐를 사용하고 train의 경우에는 4100,4110가 있어야함 아니면 pass
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        sessions_count += 1
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [ ]:
one = train[train.installation_id == '0001e90f']
two = train[train.installation_id == '0006a69f']
sample = pd.concat([one,two])

In [ ]:
two.type.value_counts()

In [ ]:
compiled_train = []
compiled_test = []
for i, (ins_id, user_sample) in tqdm(enumerate(sample.groupby('installation_id', sort = False)), total = 17000):
    compiled_train += get_data(user_sample)

In [ ]:
compiled_train